# Optional: Dropout

In this Notebook we introduce the idea of Dropout and how it aids in Neural Network training. 

By completing this exercise you will:
1. Know the implementation details of Dropout
2. Notice the difference in behavior during train and test time
3. Use Dropout in a Fully Connected Layer to see how it affects training

# What is Dropout

Dropout [1] is a technique for regularizing neural networks by randomly setting some features to zero during the forward pass. Dropout would help your Neural Network to perform better on Test data.

[1] Geoffrey E. Hinton et al, "Improving neural networks by preventing co-adaptation of feature detectors", arXiv 2012

In [1]:
# As usual, a bit of setup

import time
import numpy as np
import matplotlib.pyplot as plt
from exercise_code.layers import *
from exercise_code.tests import *
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
from exercise_code.BatchNormModel import SimpleNetwork, DropoutNetwork
import pytorch_lightning as pl
import shutil

import os
from pytorch_lightning.loggers import TensorBoardLogger

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Dropout forward pass
In the file `exercise_code/layers.py`, implement the forward pass for dropout. Since dropout behaves differently during training and testing, make sure to implement the operation for both modes.

Once you have done so, run the cell below to test your implementation.

In [2]:
x = np.random.randn(500, 500) + 10
# Let us use different dropout values(p) for our dropout layer and see their effects
for p in [0.3, 0.6, 0.75]:
    out, _ = dropout_forward(x, {'mode': 'train', 'p': p})
    out_test, _ = dropout_forward(x, {'mode': 'test', 'p': p})

    print('Running tests with p = ', p)
    print('Mean of input: ', x.mean())
    print('Mean of train-time output: ', out.mean())
    print('Mean of test-time output: ', out_test.mean())
    print('Fraction of train-time output set to zero: ', (out == 0).mean())
    print('Fraction of test-time output set to zero: ', (out_test == 0).mean())
    print()

Running tests with p =  0.3
Mean of input:  9.995692269099392
Mean of train-time output:  9.991340776836056
Mean of test-time output:  9.995692269099392
Fraction of train-time output set to zero:  0.300212
Fraction of test-time output set to zero:  0.0

Running tests with p =  0.6
Mean of input:  9.995692269099392
Mean of train-time output:  9.98293600305371
Mean of test-time output:  9.995692269099392
Fraction of train-time output set to zero:  0.600648
Fraction of test-time output set to zero:  0.0

Running tests with p =  0.75
Mean of input:  9.995692269099392
Mean of train-time output:  9.967733454211185
Mean of test-time output:  9.995692269099392
Fraction of train-time output set to zero:  0.750608
Fraction of test-time output set to zero:  0.0



# Dropout backward pass
In the file `exercise_code/layers.py`, implement the backward pass for dropout. After doing so, run the following cell to numerically gradient-check your implementation.

In [3]:
x = np.random.randn(10, 10) + 10
dout = np.random.randn(*x.shape)

dropout_param = {'mode': 'train', 'p': 0.8, 'seed': 123}
out, cache = dropout_forward(x, dropout_param)
dx = dropout_backward(dout, cache)
dx_num = eval_numerical_gradient_array(lambda xx: dropout_forward(xx, dropout_param)[0], x, dout)

print('dx relative error: ', rel_error(dx, dx_num))

dx relative error:  1.892905277373018e-11


# Fully-connected nets with Dropout
As an experiment, we will train a pair of two-layer networks on training dataset: one will use no dropout, and one will use a dropout probability of 0.75. We will then visualize the training and validation accuracies of the two networks over time.
We are going to use PyTorch Lightning Fully Connected Network and see the effects of dropout layer. Feel free to check `exercise_code/BatchNormModel.py` and play around with the parameters

## Setup TensorBoard
In exercise 07 you've already learned how to use TensorBoard. Let's use it again to make the debugging of our network and training process more convenient! Throughout this notebook, feel free to add further logs or visualizations your TensorBoard!

In [4]:
# Few Hyperparameters before we start things off
hidden_dim = 200
batch_size = 50

epochs = 5
learning_rate = 0.00005
logdir = './dropout_logs'
if os.path.exists(logdir):
    # We delete the logs on the first run
    shutil.rmtree(logdir)
os.mkdir(logdir)

In [5]:
%load_ext tensorboard
%tensorboard --logdir dropout_logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 18104.

In [6]:
# train
model = SimpleNetwork(hidden_dim=hidden_dim, batch_size=batch_size, learning_rate=learning_rate)
simple_network_logger = TensorBoardLogger(
    save_dir=logdir,
    name='simple_network'
)
trainer = pl.Trainer(max_epochs=epochs, logger=simple_network_logger)

trainer.fit(model)

GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

  | Name    | Type       | Params
-----------------------------------
0 | model   | Sequential | 159 K 
1 | model.0 | Linear     | 157 K 
2 | model.1 | ReLU       | 0     
3 | model.2 | Linear     | 2 K   
Validation sanity check: 0it [00:00, ?it/s]Val-Acc=0.0004166666666666667
Epoch 3:  80%|████████  | 960/1200 [00:24<00:06, 38.60it/s, loss=0.555, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 4:  47%|████▋     | 569/1200 [00:14<00:16, 38.40it/s, loss=0.512, v_num=0]


1

In [8]:
# train
model = DropoutNetwork(hidden_dim=hidden_dim, batch_size=batch_size, learning_rate=learning_rate,dropout_p=0.75)
dropout_network_logger = TensorBoardLogger(
    save_dir=logdir,
    name='dropout_network'
)
trainer = pl.Trainer(max_epochs=epochs, logger=dropout_network_logger)

trainer.fit(model)

GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

  | Name    | Type        | Params
------------------------------------
0 | model   | Sequential  | 159 K 
1 | model.0 | Linear      | 157 K 
2 | model.1 | BatchNorm1d | 400   
3 | model.2 | ReLU        | 0     
4 | model.3 | Dropout     | 0     
5 | model.4 | Linear      | 2 K   
Validation sanity check: 0it [00:00, ?it/s]Val-Acc=0.001
Epoch 5:  80%|████████  | 960/1200 [00:23<00:05, 41.33it/s, loss=0.572, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 5: 100%|██████████| 1200/1200 [00:26<00:00, 45.54it/s, loss=0.572, v_num=0]


1

In [9]:
%load_ext tensorboard
%tensorboard --logdir dropout_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 24784), started 0:06:44 ago. (Use '!kill 24784' to kill it.)

## Observation
As you can see, by using Dropout we would see that the Training Accuracy would decrease but the model would perform better on the Validation Set. Like Batch Normalization, Dropout also has different behavior at Train and Test time. 